In [1]:
import recordlinkage as rl
import pandas as pd

In [2]:
dataset1 = pd.read_csv('dataset1.csv')
dataset1.rename(str.strip, axis='columns', inplace=True)
dataset1.head(6)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
0,rec-223-org,,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
1,rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
2,rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
3,rec-10-dup-0,kayla,harrington,,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
4,rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
5,rec-6-dup-0,,trevorrow,16,dumas street,2/98-latchford barracks,mount immaney,2281,wa,19530313,4811642


In [3]:
print(dataset1.shape)
print(dataset1.info())

(1000, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rec_id         1000 non-null   object
 1   given_name     1000 non-null   object
 2   surname        1000 non-null   object
 3   street_number  1000 non-null   object
 4   address_1      1000 non-null   object
 5   address_2      1000 non-null   object
 6   suburb         1000 non-null   object
 7   postcode       1000 non-null   int64 
 8   state          1000 non-null   object
 9   date_of_birth  1000 non-null   object
 10  soc_sec_id     1000 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 86.1+ KB
None


## Combinación 1

- Indexado: BlockIndex
- Comparación: Jaro

In [4]:
def block_index_():
    indexer = rl.Index()
    indexer.block('given_name')
    indexer.block('surname')
    indexer.block('suburb')
    indexer.block('postcode')
    indexer.block('date_of_birth')
    indexer.block('soc_sec_id')
    compare_indices = indexer.index(dataset1)
    return compare_indices
    
def compare_(compare_indices, method):
    comp = rl.Compare()
    comp.string('given_name', 'given_name', label='given_name', method=method)
    comp.string('surname', 'surname', label='surname', method=method)
    comp.string('suburb', 'suburb', label='suburb', method=method)
    comp.string('date_of_birth', 'date_of_birth', label='date_of_birth', method=method)
    comp.numeric('postcode', 'postcode', label='postcode')
    comp.numeric('soc_sec_id', 'soc_sec_id', label='soc_sec_id')
    features = comp.compute(compare_indices, dataset1)
    return features

In [5]:
cp_idxs1 = block_index_()
features_1 = compare_(compare_indices=cp_idxs1, method='jaro')

In [6]:
features_1.head()

given_name   surname    suburb  date_of_birth  postcode  soc_sec_id
5 0    1.000000  0.465079  0.587302       0.703704       0.0         0.0
8 0    1.000000  0.588624  0.531481       0.756614       0.0         0.0
  5    1.000000  0.433333  0.504762       0.703704       0.0         0.0
9 0    1.000000  0.523810  0.481481       0.722222       0.0         0.0
  3    0.722222  0.489177  1.000000       0.842593       0.0         0.0

In [7]:
features_1.describe()

,given_name,surname,suburb,date_of_birth,postcode,soc_sec_id
count,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000
mean,0.778831,0.681526,0.621084,0.772828,0.140779,0.068192
std,0.216940,0.216482,0.176916,0.123854,0.346838,0.252094
min,0.408333,0.400673,0.361111,0.555556,0.000000,0.000000
25%,0.565079,0.513889,0.504389,0.703704,0.000000,0.000000
50%,0.722222,0.587302,0.563492,0.722222,0.000000,0.000000
75%,1.000000,1.000000,0.660185,0.804233,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Combinación 2

- Indexado: BlockIndex
- Comparación: Levenshtein

In [8]:
cp_idx2 = block_index_()
features_2 = compare_(compare_indices=cp_idx2, method='levenshtein')

In [9]:
features_2.head()

given_name   surname    suburb  date_of_birth  postcode  soc_sec_id
5 0    1.000000  0.200000  0.357143       0.333333       0.0         0.0
8 0    1.000000  0.222222  0.100000       0.444444       0.0         0.0
  5    1.000000  0.200000  0.142857       0.555556       0.0         0.0
9 0    1.000000  0.285714  0.222222       0.333333       0.0         0.0
  3    0.166667  0.181818  1.000000       0.444444       0.0         0.0

In [10]:
features_2.describe()

,given_name,surname,suburb,date_of_birth,postcode,soc_sec_id
count,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000
mean,0.598388,0.452650,0.331774,0.570794,0.140779,0.068192
std,0.382355,0.360652,0.294099,0.245203,0.346838,0.252094
min,0.076923,0.083333,0.055556,0.111111,0.000000,0.000000
25%,0.222222,0.166667,0.153846,0.444444,0.000000,0.000000
50%,0.428571,0.272727,0.222222,0.555556,0.000000,0.000000
75%,1.000000,1.000000,0.333333,0.666667,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Combinación 3
- Indexado: BlockIndex
- Comparación: Q-gram

In [12]:
cp_idx3 = block_index_()
features_3 = compare_(compare_indices=cp_idx3, method='qgram')

In [13]:
features_3.head()

given_name  surname    suburb  date_of_birth  postcode  soc_sec_id
5 0         0.0      0.0  0.071429       0.222222       0.0         0.0
8 0         0.0      0.0  0.000000       0.444444       0.0         0.0
  5         0.0      0.0  0.000000       0.222222       0.0         0.0
9 0         0.0      0.0  0.000000       0.222222       0.0         0.0
  3         0.0      0.0  1.000000       0.222222       0.0         0.0

In [14]:
features_3.describe()

,given_name,surname,suburb,date_of_birth,postcode,soc_sec_id
count,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000
mean,0.346595,0.296393,0.175103,0.297301,0.140779,0.068192
std,0.452538,0.428330,0.320718,0.240344,0.346838,0.252094
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.222222,0.000000,0.000000
50%,0.000000,0.000000,0.055556,0.222222,0.000000,0.000000
75%,1.000000,1.000000,0.125000,0.333333,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
